In [1]:
# GOTCHA query requires 'plate' but FITS requires 'plateid'

In [1]:
import urllib.request

import numpy as np

import pickle

import matplotlib.pyplot as plt
%matplotlib inline

from astropy.io import fits
from astropy.utils.data import download_file

from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [2]:
#download and return a handle to the FITS file spceified by the function arguments

def get_fits (plateid, mjd, fiberid):
    v = list(map(str,(plateid, mjd, fiberid)))
    preamble = 'http://dr15.sdss.org/optical/spectrum/view/data/format=fits?'
    # this is mysterious, but it's in the example and works
    postamble = '&reduction2d=v5_7_0'
    input_file =  download_file(preamble + 'plateid=' + v[0] + '&mjd=' + v[1] + '&fiberid=' + v[2] + postamble, cache=True )
    return (input_file)
                   
#preamble='http://dr15.sdss.org/optical/spectrum/view/data/format=fits?'

In [3]:
sampleQuery = ("SELECT TOP 10 specObjID, plate, mjd, fiberid, z FROM SpecObj WHERE class = 'QSO' AND zWarning = 0")

In [4]:
preamble = "http://skyserver.sdss.org/dr15/en/tools/search/x_results.aspx?searchtool=SQL&format=csv&cmd="

# there ought to be accommodation here for error checking, new lines, comments and so on

payload = sampleQuery.replace(' ', '+')

In [5]:
with urllib.request.urlopen(preamble + payload) as response:
   csv = response.read()

In [6]:
# csv is actually a bytes object, so we need to decode it
# split it into lines while we're here
outList = (csv.decode("utf-8")).split('\n')

In [7]:
# discard first and final rows, store the rest
# dataFields holds column names
# dataList has the numbers
dataFields = outList[1].split(',')
dataList = outList[2:-1]

In [8]:
# a list comprehension! split each string in dataList into a list

dataTable = [x.split(',') for x in dataList]

In [9]:
def get_spectrum (plateID, mjd, fiberid):
    
    #get and open the file

    hdu_list = fits.open(get_fits(plateID, mjd, fiberid))
    # spectrum data is in COADD which is HDU1
    # https://data.sdss.org/datamodel/files/BOSS_SPECTRO_REDUX/RUN2D/spectra/PLATE4/spec.html
    # hdu_list[1] returns a recarray, so we can apply labels

    spectrum_data = np.array(hdu_list[1].data,dtype=[('flux', 'f4'),('loglam', 'f4'), ('ivar', 'f4'), ('and_mask', 'i4'), ('or_mask', 'i4'), \
                                                    ('wdisp', 'f4'),('sky', 'f4'), ('model', 'f4')])
  
    hdu_list.close()
    
    return (spectrum_data)

In [10]:
def write_linear_restframe_spectrum_file (specObjID, spectrum, z):
    flux = spectrum['flux']
    wavelength = np.power(10.0,spectrum['loglam'])
    wavelength = wavelength / (1.0 + float(z))
    ivar = spectrum['ivar']
    and_mask = spectrum['and_mask']
    or_mask = spectrum['or_mask']
    
    with open('./Spectra/' + specObjID + '.spec', 'wb') as filehandle:  
        # store the data as binary data stream
        pickle.dump([wavelength, flux, ivar, and_mask, or_mask], filehandle)

    

In [11]:
def read_linear_restframe_spectrum_file (specObjID):
    
    with open('./Spectra/' + specObjID + '.spec', 'rb') as filehandle:  
        # read the data as binary data stream
        [wavelength, flux, ivar, and_mask, or_mask] = pickle.load(filehandle)
    return [wavelength, flux, ivar, and_mask, or_mask]

In [12]:
count = 0
total = len (dataTable)
# to draw seperate plots, use a list of figures and axes
fig = []
ax = []

for object in dataTable:
    
    # check we're making progress
    count +=1
    print ("processing object " + str(count) + " of " + str(total) + ", " + object[0])
    
    spectrum = get_spectrum (object[1],object[2], object[3])
    write_linear_restframe_spectrum_file (object[0], spectrum, object[4])
    
    # and_flag all good is &000 
    # testing with mod 8 is a quick and dirty way to check this
    flags = (spectrum['and_mask']%8)
    if (np.any(flags != 0)):
        print (object[0] + " has " + str(len(np.nonzero(flags))) + " bad data flagged")
        print (np.nonzero (flags))
    
    
    # rip flux and wavelength, changing the latter from log to linear
  
    flux = spectrum['flux']
    wavelength = np.power(10.0,spectrum['loglam'])
    wavelength = wavelength / (1.0 + float(object[4]))
    
    #quick and dirty plot just to check it looks right

    p = figure(title="SpecObjID " + object[0], plot_height=350, plot_width=800)
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Wavelength'
    p.yaxis.axis_label = 'Flux'

    p.line(wavelength, flux)

    show(p)
    
    #fig.append(plt.figure(count))
    #plt.figure(count).suptitle("SpecObjID " + object[0])
    #ax.append(fig[count-1].gca())
    #ax[count-1].plot(wavelength, flux)

processing object 1 of 10, 299490227200747520


processing object 2 of 10, 299505345485629440


processing object 3 of 10, 299511392799582208


processing object 4 of 10, 299523762305394688


processing object 5 of 10, 299524037183301632


processing object 6 of 10, 299525961328650240


processing object 7 of 10, 299533657910044672


processing object 8 of 10, 299544378148415488


processing object 9 of 10, 299546577171671040


processing object 10 of 10, 299551250096089088


In [33]:
print (flags[1541])

0


In [32]:
print(flags)

[0 0 0 ... 0 0 0]
